In [8]:
import PyPDF2
import camelot
import pandas as pd

In [9]:
#FAILED PYPDF IMPLEMENTATION

# text = ""
# with open("engg-credit-transfer-database.pdf", 'rb') as file:
#     reader = PyPDF2.PdfReader(file)
#     for page in reader.pages:
#         text += page.extract_text()

# with open("tester.txt", "w+") as tester_file:
#     tester_file.write(text)

In [10]:
engg_file = open("engg-credit-transfer-database.pdf", 'rb')

#store data in pdfReader
pdfReader = PyPDF2.PdfReader(engg_file)

#count number of pages
totalPages = len(pdfReader.pages)
totalPages

39

In [11]:
engg_pdf = camelot.read_pdf("engg-credit-transfer-database.pdf", 
                       pages='1-end') #address of file location
initial_df = pd.DataFrame()

#add data to DF
for i in range(totalPages + 1):
    if i == 0: initial_df = engg_pdf[i].df
    else: initial_df = pd.concat([initial_df, engg_pdf[i].df], ignore_index=True)


In [12]:
initial_df.columns = ['BEng_Prog',
                    'University',
                    'University-Country',
                    'Exchange-Course-Code',
                    'Exchange-Course-Title',
                    'HKU-Course-Code',
                    'HKU-Course-Title',
                    'Remarks']

In [13]:
#cleaning if it is null, blank, or if it's a header
cleaned_df = initial_df[initial_df['University'].str.strip().ne('') & 
                        (initial_df['University'] != 'University') & (initial_df['University'].notna())]

#elective/discipline elective distinction
def set_course_code(row):
    if row['HKU-Course-Code'] == '':
        if 'discipline' in row['HKU-Course-Title'].lower():
            return row['BEng_Prog'] + 'XXXX'
        else:
            return 'ELECTIVE'
    return row['HKU-Course-Code']  # Return the existing value if not blank

# apply the function
cleaned_df['HKU-Course-Code'] = cleaned_df.apply(set_course_code, axis=1)

# encoding issues
cleaned_df['University'] = cleaned_df['University'].str.replace(
    "Queen‚Äôs University, Kingston", 
    "Queen's University, Kingston", 
    regex=False
)

/var/folders/nt/67mjll3s4fb1vnp8mvn0q2kh0000gn/T/ipykernel_36927/1441412203.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['HKU-Course-Code'] = cleaned_df.apply(set_course_code, axis=1)
/var/folders/nt/67mjll3s4fb1vnp8mvn0q2kh0000gn/T/ipykernel_36927/1441412203.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['University'] = cleaned_df['University'].str.replace(


In [14]:
#dropped for unification
cleaned_df = cleaned_df.drop(columns=["BEng_Prog", "Remarks"])
cleaned_df[:5]

,University,University-Country,Exchange-Course-Code,Exchange-Course-Title,HKU-Course-Code,HKU-Course-Title
6,Griffith University,Australia,2201ENG,Engineering Thermodynamics,MECH3402,Engineering thermodynamics
7,Griffith University,Australia,3004ENG,Project Management Principles,MECH4410,Engineering & technology management
8,Griffith University,Australia,3319ENG,Power Transmission and Distribution,MECH3406,Electrical and electronic engineering
9,Griffith University,Australia,3505ENG,Heat and Mass Transfer,MECH4411,Heat transfer
10,Griffith University,Australia,3511ENG,Design of Machine Elements,MECH3427,Design and manufacture


In [15]:
# FOR CHECKING PURPOSES, and exporting
# cleaned_df.iloc[716]
cleaned_df.to_csv("engg-credit_transfer_database.csv",index=False)